# Plotting a pourbaix diagram using the legacy Materials Project API

**NOTE** this notebook uses the LEGACY Materials Project API, which is frozen at database release `v2021.03.13`. For an updated experience and access to the latest data, we recommend the new API. See [Materials Project API reference](https://next-gen.materialsproject.org/api) for more information.

*If you use this infrastructure, please consider citing the following work:*

[K. A. Persson, B. Waldwick, P. Lazic and G. Ceder, Phys. Rev. B, 2012, 85, 235438.](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.235438)

[A. K. Singh, L. Zhou, A. Shinde, S. K. Suram, J. H. Montoya, D. Winston, J. M. Gregoire and K. A. Persson, Chem. Mater., 2017, DOI: acs.chemmater.7b03980.](http://pubs.acs.org/doi/10.1021/acs.chemmater.7b03980)

**Notebook Author:** Joseph Montoya, Ryan Kingsbury

**Last Update** 2023-03-10

----------

The Materials Project REST interface includes functionality to construct pourbaix diagrams from computed entries.  Note that the Pourbaix infrastructure is still undergoing revision, but now includes a simplified interface that enables MP and pymatgen users to fetch entries that have been processed according to the [Materials Project Aqueous Compatibility scheme](https://docs.materialsproject.org/methodology/materials-methodology/aqueous-stability-pourbaix).  Thus, users can reproduce web Pourbaix diagrams in two or three steps in pymatgen.

In [ ]:
# Uncomment the subsequent lines in this cell to install dependencies for Google Colab.
# !pip install pymatgen==2022.7.19
from __future__ import annotations

**NOTE** - make sure to paste your **legacy** API key below.

In [ ]:
MP_API_KEY = "<your-legacy-API-key-here>"

In [ ]:
# Import necessary tools from pymatgen
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter

%matplotlib inline

# Initialize the MP Rester
mpr = MPRester()

To retrieve entries necessary to construct a Pourbaix Diagram use `MPRester.get_pourbaix_entries(LIST_OF_ELEMENTS)` with a list of elements comprising your chemical system.  It is not necessary to include 'O' and 'H' in your list, as they are added automatically.  This function also contains all of necessary preprocessing to ensure the PourbaixEntries are compatible with the pymatgen `PourbaixDiagram` constructor.

In [ ]:
# Get all pourbaix entries corresponding to the Cu-O-H chemical system.
entries = mpr.get_pourbaix_entries(["Cu"])

/Users/shyue/miniconda3/envs/mavrl/lib/python3.11/site-packages/mp_api/client/mprester.py:368: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'query_contributions'

Pourbaix diagrams can be constructed using `PourbaixDiagram(RETRIEVED_ENTRIES)` as below.  Note that a `comp_dict` keyword argument may also be supplied to the `PourbaixDiagram` constructor if a fixed composition for a multi-element pourbaix diagram is desired.

In [ ]:
# Construct the PourbaixDiagram object
pbx = PourbaixDiagram(entries)

The `PourbaixDiagram` includes a number of useful functions for determining stable species and stability of entries relative to a given pourbaix facet (i.e. as a function of pH and V).

In [ ]:
# Get an entry stability as a function of pH and V
entry = [e for e in entries if e.entry_id == "mp-1692"][0]
print(
    "CuO's potential energy per atom relative to the most",
    f"stable decomposition product is {pbx.get_decomposition_energy(entry, pH=7, V=-0.2):0.2f} eV/atom",
)

This suggests that CuO, for example, has a large driving force for decomposition at neutral pH and mildly reducing conditions.

To see this in more detail, we can plot the pourbaix diagram.  The `PourbaixPlotter` object is also initialized using an instance of the `PourbaixDiagram` object.

In [ ]:
plotter = PourbaixPlotter(pbx)

In [ ]:
plotter.get_pourbaix_plot().show()

You can customize the appearance of the plot using keyword arguments

In [ ]:
plotter.get_pourbaix_plot(
    limits=[[0, 14], [-2, 2]],  # pH and E limits
    title="Customized Pourbaix Plot",
    label_domains=True,  # set True to label phases
    label_fontsize=10,  # font size for the phase labels
    show_neutral_axes=False,  # show or hide axes at pH=0, E=0
    show_water_lines=False,  # show or hide lines for water stability
)

The PourbaixPlotter object can also plot the relative stability of an entry across the pourbaix diagram.  To do this, use the `PourbaixPlotter.plot_entry_stability` method.

In [ ]:
plt = plotter.plot_entry_stability(entry)

# Plotting k-nary systems

Pymatgen also supports binary/ternary pourbaix diagrams with fixed compositions of the non-H or O elements.  This is achieved by finding all possible combinations of entries that fulfill the composition constraint and treating them as individual entries in pourbaix space.  Note that you can supply a composition dictionary to further tune the pourbaix diagram.

In [ ]:
# Get all pourbaix entries corresponding to the Bi-V-O-H chemical system.
entries = mpr.get_pourbaix_entries(["Bi", "V"])
# Construct the PourbaixDiagram object
pbx = PourbaixDiagram(
    entries,
    comp_dict={"Bi": 0.5, "V": 0.5},
    conc_dict={"Bi": 1e-8, "V": 1e-8},
    filter_solids=True,
)

Note that the `filter_solids` keyword argument in the `PourbaixDiagram` instantiation above tells the constructor whether to filter solids by phase stability on the compositional phase diagram for solids. The filtering process significantly reduces the time it takes to generate all of the combined entries for a binary or ternary pourbaix diagram though, so it may be prudent to use in those cases. Pourbaix Diagrams generated with and without this argument may look significantly different in the OER and HER regions, since highly oxidized materials (e. g. Bi$_2$O$_5$) or highly reduced materials (e. g. most hydrides) may not be stable on the compositional phase diagram. 

In [ ]:
# Construct the pourbaix analyzer
plotter = PourbaixPlotter(pbx)
plt = plotter.get_pourbaix_plot()
plt.show()

Getting the heatmaps for a solid entry in these cases is a bit more involved, because many of the regions of the pourbaix diagram include multiphase entries.  Here's an example for this case.

In [ ]:
bivo4_entry = [entry for entry in entries if entry.entry_id == "mp-545850"][0]
plt = plotter.plot_entry_stability(bivo4_entry)

If a compound is provided that doesn't meet the constraints of the composition (in this case, equal parts Bi and V), you will get a `ValueError`

In [ ]:
bio2_entry = [entry for entry in entries if entry.entry_id == "mp-557993"][0]
plt = plotter.plot_entry_stability(bio2_entry)